In [9]:
# import sys
# try:
#   import google.colab
#   IN_COLAB = True
# except:
#   IN_COLAB = False

# if IN_COLAB:
#     # montar el drive, que es donde tenemos el dataset
#     from google.colab import drive
#     drive.mount("/content/drive")
#     data_dir = "/content/drive/MyDrive/Colab Notebooks/datasets/"
#     sys.path.append(data_dir)
# else:
#     import os
#     data_dir = os.path.dirname(__file__) + "/"

In [10]:
import numpy as np
np.random.seed(5)

from keras.layers import Input, Dense, SimpleRNN
from keras.models import Model
from keras.optimizers import SGD
from keras.utils import to_categorical

from keras import backend as K
import tensorflow as tf
from tensorflow import keras

In [11]:
# 1. LECTURA DEL SET DE DATOS
# ===========================================================
nombres = open('archives/nombres_dinosaurios.txt','r').read()
nombres = nombres.lower()

# Crear diccionario (listado de caracteres que no se repiten)
alfabeto = list(set(nombres))
tam_datos, tam_alfabeto = len(nombres), len(alfabeto)
print("En total hay %d caracteres, y el diccionario tiene un tamaño de %d caracteres." % (tam_datos, tam_alfabeto))

# Conversión de caracteres a índices y viceversa
car_a_ind = { car:ind for ind,car in enumerate(sorted(alfabeto))}
ind_a_car = { ind:car for ind,car in enumerate(sorted(alfabeto))}
print(car_a_ind)
print(ind_a_car)

En total hay 19909 caracteres, y el diccionario tiene un tamaño de 27 caracteres.
{'\n': 0, 'a': 1, 'b': 2, 'c': 3, 'd': 4, 'e': 5, 'f': 6, 'g': 7, 'h': 8, 'i': 9, 'j': 10, 'k': 11, 'l': 12, 'm': 13, 'n': 14, 'o': 15, 'p': 16, 'q': 17, 'r': 18, 's': 19, 't': 20, 'u': 21, 'v': 22, 'w': 23, 'x': 24, 'y': 25, 'z': 26}
{0: '\n', 1: 'a', 2: 'b', 3: 'c', 4: 'd', 5: 'e', 6: 'f', 7: 'g', 8: 'h', 9: 'i', 10: 'j', 11: 'k', 12: 'l', 13: 'm', 14: 'n', 15: 'o', 16: 'p', 17: 'q', 18: 'r', 19: 's', 20: 't', 21: 'u', 22: 'v', 23: 'w', 24: 'x', 25: 'y', 26: 'z'}


In [12]:
nneuronas = 128
model = keras.models.Sequential([
    keras.layers.SimpleRNN(nneuronas, return_sequences=True,
                           input_shape=(None, tam_alfabeto)),
    keras.layers.SimpleRNN(nneuronas, return_sequences=True),
    keras.layers.Dense(tam_alfabeto, activation="softmax")
])
model.compile(loss="categorical_crossentropy", optimizer="adam")
model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ simple_rnn_2 (SimpleRNN)        │ (None, None, 128)      │        19,968 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ simple_rnn_3 (SimpleRNN)        │ (None, None, 128)      │        32,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, None, 27)       │         3,483 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 56,347 (220.11 KB)

 Trainable params: 56,347 (220.11 KB)

 Non-trainable params: 0 (0.00 B)

In [13]:
keras.utils.plot_model(model, "dinos.png", show_shapes=True)

You must install pydot (`pip install pydot`) for `plot_model` to work.


In [14]:
with open("archives/nombres_dinosaurios.txt") as f:
    ejemplos = f.readlines()
ejemplos = [x.lower().strip() for x in ejemplos]

In [15]:
def genera_train():
    rx = []
    ry = []
    ejemplo = ejemplos[np.random.randint(0, len(ejemplos))]
    X = [None] + [car_a_ind[c] for c in ejemplo]
    Y = X[1:] + [car_a_ind["\n"]]
    onehot = to_categorical(X[1:], tam_alfabeto)
    x = np.zeros((len(X), tam_alfabeto))
    x[1:] = onehot.astype("float32")
    y = to_categorical(Y, tam_alfabeto).reshape(len(X), tam_alfabeto)
    rx.append(x)
    ry.append(y)
    return np.array(rx), np.array(ry)

In [16]:
niters = 1000
for j in range(niters):
    rx, ry = genera_train()
    historia = model.train_on_batch(rx, ry)
    if j%100 == 0: print(f"Iter: {j} loss: {historia}")

Iter: 0 loss: 3.3510124683380127
Iter: 100 loss: 2.513373851776123
Iter: 200 loss: 2.3573315143585205
Iter: 300 loss: 2.2978711128234863
Iter: 400 loss: 2.2578601837158203
Iter: 500 loss: 2.23634934425354
Iter: 600 loss: 2.2041220664978027
Iter: 700 loss: 2.181370735168457
Iter: 800 loss: 2.1443049907684326
Iter: 900 loss: 2.118657112121582


In [17]:
def genera_nombre():
    x = np.zeros((1,1,tam_alfabeto))
    nombre = ""
    car = -1
    count = 0
    while car != "\n":
        pred = model.predict(x, verbose=0)
        ultimo = pred[0:, -1:, :]
        ix = np.random.choice(list(range(tam_alfabeto)),
                              p=ultimo.ravel())
        car = ind_a_car[ix]
        nombre += car
        x1 = to_categorical(ix, tam_alfabeto)
        x = x.reshape(-1, 27)
        x1 = x1.reshape(-1, 27)
        x = np.append(x, x1)
        x = x.reshape(1, -1, 27)
        count += 1

    print(nombre)

In [18]:
for i in range(20):
    genera_nombre()

galtodia

fcerldopusus

hygcerotonusus

aurivos

nucoociten

emvitateus

byleggaur

dexisaunus

jronongsas

saurusauruh

tuauruo

kungtyttos

rhbfodis

bheodon

hondadanalus

tyina

xebattong

euteradestosuu

eutorus

lantolus

